In [2]:
import pandas as pd
import numpy as np
import json
import re
from pathlib import Path

In [20]:
MAIN_PATH = Path(input()) # Input root directory

In [22]:
# with open(MAIN_PATH / r"wiki_crawling\txt_dict.json", 'r') as file:
#     txt1 = json.load(file)

with open(MAIN_PATH / 'wiki_crawling/txt_dict.json', 'r') as file:
    txt1 = json.load(file)
    
with open(MAIN_PATH / 'wiki_people/wiki_crawling/txt_dict_2.json', 'r') as file:
    txt2 = json.load(file)

In [ ]:
txt1['Doja Cat']['Early life']['']

In [24]:
from nltk import sent_tokenize

In [ ]:
sent_tokenize(txt1['Doja Cat']['Early life'][''])

In [29]:
def year_extraction(sentence_list):
    year_sentence_list = []
    
    for x in sentence_list:
        if re.search(r'\b\d{4}\b', x):
            year_sentence_list += [x]
            
    return year_sentence_list

In [34]:
import copy

txt1_dup = copy.deepcopy(txt1)
# txt2_dup = copy.deepcopy(txt2)

In [35]:
err_lst = []

for k1, v1 in txt1_dup.items():
    for k2, v2 in v1.items():
        if type(v2)==str:
            v1[k2] = year_extraction(sent_tokenize(v2))
        else:
            for k3, v3 in v2.items():
                try:
                    v2[k3] = year_extraction(sent_tokenize(v3))
                except:
                    err_lst += [k1, k2, k3, v3]

In [36]:
err_lst

[]

In [71]:
err_lst2 = []

for k1, v1 in txt2_dup.items():
    for k2, v2 in v1.items():
        if type(v2)==str:
            v1[k2] = year_extraction(sent_tokenize(v2))
        else:
            for k3, v3 in v2.items():
                try:
                    v2[k3] = year_extraction(sent_tokenize(v3))
                except:
                    err_lst2 += [k1, k2, k3, v3]

In [72]:
err_lst2

[]

In [ ]:
with open('txt1_years_dict.json', 'w') as file:
    json.dump(txt1_dup, file)
    
with open('txt2_years_dict.json', 'w') as file:
    json.dump(txt2_dup, file)

### From here is Haein's addition
- converting the json file into dataframe

In [76]:
with open(MAIN_PATH / "event_extraction/txt1_years_dict.json", "r") as json_file:
    year_json_1 = json.load(json_file)
with open(MAIN_PATH / "event_extraction/txt2_years_dict.json", "r") as json_file:
    year_json_2 = json.load(json_file)

In [46]:
def only_year_extraction(year_sentence):
    match = re.search(r'\b\d{4}\b', year_sentence)
    year = match.group(0)
    return year

In [73]:
def construct_year_df(year_json):
    people_dict = {}
    for person, person_content in year_json.items():
        person_list = []
        for big_topic, big_content in person_content.items():
            if isinstance(big_content, list):
                for sentence in big_content:
                    year = only_year_extraction(sentence)
                    row = pd.Series([year, sentence, big_topic, mid_topic])
                    person_list.append(row)
            else:
                for mid_topic, mid_content in big_content.items():
                    if isinstance(mid_content, list):
                        for sentence in mid_content:
                            year = only_year_extraction(sentence)
                            row = pd.Series([year, sentence, big_topic, mid_topic])
                            person_list.append(row)
                    if not mid_topic:
                        if not mid_content:
                            continue
                        else:
                            for sentence in mid_content:
                                year = only_year_extraction(sentence)
                                row = pd.Series([year, sentence, big_topic, mid_topic])
                                person_list.append(row)
                    else:
                        for sentence in mid_content:
                            year = only_year_extraction(sentence)
                            row = pd.Series([year, sentence, big_topic, mid_topic])
                            person_list.append(row)
        people_dict[person] = pd.DataFrame(person_list)
    return people_dict

In [78]:
people_dict_1 = construct_year_df(year_json_1)
people_dict_2 = construct_year_df(year_json_2)

In [83]:
people_dict_1

{'Michael B. Jordan':        0                                                  1              2  \
 0   1987   Michael Bakari Jordan (/bɑːˈkɑːri/ bah-KAR-ee...        summary   
 1   2013  He is best known for his film roles as shootin...        summary   
 2   2018  Jordan reprised his role of Creed in Creed II ...        summary   
 3   2002  Jordan initially broke out in television, play...        summary   
 4   2003  He went on to play Reggie Montgomery on the AB...        summary   
 ..   ...                                                ...            ...   
 83  2018  As of 2018[update], he lives with his parents ...  Personal life   
 84  2020  In November 2020, the actor began dating model...  Personal life   
 85  2022  In June 2022, it was announced that the couple...  Personal life   
 86  2022   In December 2022, Jordan was announced as par...  Personal life   
 87  2022   In December 2022, Jordan was announced as par...  Personal life   
 
                    3  
 0   

In [82]:
people_dict_2['Winston Churchill'].to_csv("Winston_Churchill.csv")